<a href="https://colab.research.google.com/github/Shun0212/CameraAndTranslateGPTApp/blob/master/CodeHawks_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U transformers>=4.48.0
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 897.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.0 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
from huggingface_hub import login

# Colab のシークレット環境変数から Hugging Face トークンを取得
huggingface_token = userdata.get("HUGGINGFACE_TOKEN")

if huggingface_token:
    login(huggingface_token)
    print("ログイン成功")
else:
    print("HUGGINGFACE_TOKEN が設定されていません。")


ログイン成功


In [3]:
from transformers import AutoModel, AutoTokenizer
import os
import sys
import torch
import random
from datasets import load_dataset, Dataset
from tokenizers import BertWordPieceTokenizer
from transformers import ModernBertConfig, ModernBertForMaskedLM, PreTrainedTokenizerFast, DataCollatorForLanguageModeling, Trainer, TrainingArguments


# モデル名
repo_name = "Shuu12121/CodeMorph-ModernBERT"
# Hugging Face からモデルをロード
model = ModernBertForMaskedLM.from_pretrained(repo_name)
tokenizer = AutoTokenizer.from_pretrained(repo_name)

print("モデルのロード成功！")
print(model)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用デバイス: {device}")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/609M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/408k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

モデルのロード成功！
ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50000, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=3072, out_features=768, bias=False)
        )
      )
     

ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50000, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=3072, out_features=768, bias=False)
        )
      )
      (1-11): 11

In [ ]:
from transformers import pipeline

fill_mask = pipeline("fill-mask", model=model, tokenizer=tokenizer)
print(fill_mask("def add_numbers(a, b): return a + [MASK]"))

Device set to use cuda:0


[{'score': 0.9981516003608704, 'token': 49908, 'token_str': 'b', 'sequence': 'def add _ numbers ( a, b ) : return a + b'}, {'score': 0.0010269464692100883, 'token': 49886, 'token_str': 'a', 'sequence': 'def add _ numbers ( a, b ) : return a + a'}, {'score': 0.00034104351652786136, 'token': 49891, 'token_str': 'c', 'sequence': 'def add _ numbers ( a, b ) : return a + c'}, {'score': 8.821619121590629e-05, 'token': 49938, 'token_str': '1', 'sequence': 'def add _ numbers ( a, b ) : return a + 1'}, {'score': 2.8221440516063012e-05, 'token': 159, 'token_str': 'end', 'sequence': 'def add _ numbers ( a, b ) : return a + end'}]


In [4]:
import torch

def get_embedding(text, model, tokenizer, device="cuda"):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256)
    # token_type_ids があれば削除する
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.model(**inputs)
    embedding = outputs.last_hidden_state[:, 0, :]
    return embedding

embedding = get_embedding("def my_function(): pass", model, tokenizer)
print(embedding.shape)


torch.Size([1, 768])


In [8]:
import torch
import numpy as np
import random
import re
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer, T5ForConditionalGeneration
from sklearn.metrics.pairwise import cosine_similarity

# ---------------------------------------------------
# ユーティリティ関数群
# ---------------------------------------------------

def remove_multiline_strings(code):
    """
    Pythonコード中のマルチライン文字列（""" """ や ''' '''）を削除します。
    """
    pattern = r'("""(.*?)"""|\'\'\'(.*?)\'\'\')'
    return re.sub(pattern, '', code, flags=re.DOTALL)

def get_cls_embedding(model, tokenizer, text, device, max_length=256):
    """
    入力テキストの埋め込みを取得する関数。
    - BERT/CodeBERT系の場合は CLS トークンの埋め込みを使用。
    - CodeT5系の場合は encoder 出力の平均プーリングを使用。
    - ModernBERT のように内部に model がある場合は model.model を利用。
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    # ModernBERT の場合、token_type_ids が不要なので削除する
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if hasattr(model, "model"):
        outputs = model.model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "bert"):
        outputs = model.bert(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "roberta"):
        outputs = model.roberta(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "encoder"):
        # CodeT5系の場合は、CLSトークンが存在しないため平均プーリングを使用
        outputs = model.encoder(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1)
    else:
        outputs = model(**inputs)
        if hasattr(outputs, "last_hidden_state"):
            embedding = outputs.last_hidden_state[:, 0, :]
        elif hasattr(outputs, "hidden_states"):
            embedding = outputs.hidden_states[-1][:, 0, :]
        else:
            raise ValueError("モデル出力にlast_hidden_stateまたはhidden_statesがありません。")

    return embedding.detach().cpu().numpy()


def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def calculate_dcg(ranking):
    dcg = 0.0
    for i, rel in enumerate(ranking):
        dcg += rel / np.log2(i + 2)
    return dcg

def calculate_metrics(sim_matrix, k_values=[1, 5, 10, 50, 100]):
    """
    候補プール内で正解（常にインデックス0が正解と仮定）に基づいて評価指標を計算します。
    """
    num_queries = sim_matrix.shape[0]
    metrics = {
        "mrr": 0.0,
        "map": 0.0,
        "r_precision": 0.0,
        "recall@k": {k: 0.0 for k in k_values},
        "precision@k": {k: 0.0 for k in k_values},
        "ndcg@k": {k: 0.0 for k in k_values},
        "f1@k": {k: 0.0 for k in k_values},
        "success_rate@k": {k: 0.0 for k in k_values},
        "query_coverage@k": {k: 0.0 for k in k_values},
    }

    for i in range(num_queries):
        sims = sim_matrix[i]
        ranked_indices = np.argsort(-sims)
        # 正解は候補プール内の先頭（インデックス0）
        correct_rank = np.where(ranked_indices == 0)[0][0] + 1
        metrics["mrr"] += 1.0 / correct_rank
        metrics["map"] += 1.0 / correct_rank
        metrics["r_precision"] += 1.0 if correct_rank == 1 else 0.0

        for k in k_values:
            if correct_rank <= k:
                prec = 1.0 / correct_rank
                rec = 1.0
            else:
                prec = 0.0
                rec = 0.0
            f1 = calculate_f1(prec, rec)
            metrics["precision@k"][k] += prec
            metrics["recall@k"][k] += rec
            metrics["f1@k"][k] += f1

            ideal_ranking = [1.0] + [0.0] * (k - 1)
            actual_ranking = [1.0 if j == 0 else 0.0 for j in ranked_indices[:k]]
            idcg = calculate_dcg(ideal_ranking)
            dcg = calculate_dcg(actual_ranking)
            metrics["ndcg@k"][k] += dcg / idcg if idcg > 0 else 0.0

            metrics["success_rate@k"][k] += 1.0 if correct_rank <= k else 0.0
            metrics["query_coverage@k"][k] += 1.0 if correct_rank <= k else 0.0

    metrics["mrr"] /= num_queries
    metrics["map"] /= num_queries
    metrics["r_precision"] /= num_queries
    for k in k_values:
        metrics["precision@k"][k] /= num_queries
        metrics["recall@k"][k] /= num_queries
        metrics["ndcg@k"][k] /= num_queries
        metrics["f1@k"][k] /= num_queries
        metrics["success_rate@k"][k] /= num_queries
        metrics["query_coverage@k"][k] /= num_queries
    return metrics

def display_code_search_results(metrics, model_name):
    """
    評価結果を見やすいテーブル形式で表示します。
    """
    print(f"\n==== {model_name} Code Search Evaluation Results ====")
    print(f"MRR:         {metrics['mrr']:.4f}")
    print(f"MAP:         {metrics['map']:.4f}")
    print(f"R-Precision: {metrics['r_precision']:.4f}\n")

    header = "{:<6}{:<10}{:<10}{:<10}{:<10}{:<15}{:<15}"
    print(header.format("K", "Recall", "Precision", "NDCG", "F1", "Success Rate", "Query Cov."))
    print("-" * 76)
    for k in sorted(metrics["recall@k"].keys()):
        recall = metrics["recall@k"][k]
        precision = metrics["precision@k"][k]
        ndcg = metrics["ndcg@k"][k]
        f1 = metrics["f1@k"][k]
        success_rate = metrics["success_rate@k"][k]
        query_cov = metrics["query_coverage@k"][k]
        print(header.format(k,
                            f"{recall:.4f}",
                            f"{precision:.4f}",
                            f"{ndcg:.4f}",
                            f"{f1:.4f}",
                            f"{success_rate:.4f}",
                            f"{query_cov:.4f}"))

def evaluate_code_search(model, tokenizer, dataset, device, max_examples=100, pool_size=100,
                         query_field="docstring", code_field="code"):
    """
    指定したデータセット上でコード検索タスクを実施し、評価指標を計算します。
    各サンプルに対して、正解は候補プール内の先頭（自身）と仮定しています。
    """
    num_examples = min(len(dataset), max_examples)
    all_codes = [remove_multiline_strings(dataset[i][code_field]) for i in range(num_examples)]
    queries = [dataset[i][query_field] for i in range(num_examples)]

    # 各コードの埋め込み計算
    all_code_embeddings = []
    print("データセット全体のコード埋め込みを計算中...")
    for code in all_codes:
        emb = get_cls_embedding(model, tokenizer, code, device)
        all_code_embeddings.append(emb)
    all_code_embeddings = np.concatenate(all_code_embeddings, axis=0)
    print("コード埋め込み計算完了.")

    # 各クエリに対し候補プール内の類似度計算
    sim_matrices = []
    print("候補コードプールを作成し、類似度計算中...")
    for i in range(num_examples):
        query_embedding = get_cls_embedding(model, tokenizer, queries[i], device).reshape(1, -1)
        candidate_indices = [i] + random.sample([j for j in range(num_examples) if j != i], pool_size - 1)
        candidate_embeddings = all_code_embeddings[candidate_indices]
        sims = cosine_similarity(query_embedding, candidate_embeddings)[0]
        sim_matrices.append(sims)
    sim_matrix = np.array(sim_matrices)
    print("類似度計算完了.")

    metrics = calculate_metrics(sim_matrix)
    return metrics

# ---------------------------------------------------
# メイン処理：各モデル・各言語でコード検索実験を実施
# ---------------------------------------------------

if __name__ == "__main__":
    # Python の乱数シードを設定（候補サンプルの抽出などに影響します）
    random.seed(35)
    np.random.seed(35)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用デバイス: {device}")

    # ------------------------------
    # ① ModernBERT 単体ロード確認
    # ------------------------------
    repo_name = "Shuu12121/CodeMorph-ModernBERT"
    model_demo = AutoModelForMaskedLM.from_pretrained(repo_name)
    tokenizer_demo = AutoTokenizer.from_pretrained(repo_name)
    model_demo.to(device)
    print("\n【ModernBERT 単体ロード確認】")
    print("モデルのロード成功！")
    print(model_demo)

    # ------------------------------
    # ② 評価データセットの言語一覧（例）
    # ------------------------------
    languages = ["python", "java", "javascript", "php", "ruby", "go"]
    max_examples = 1000  # 各言語ごとに先頭100サンプルを利用（ランダムサンプリング後）

    # ------------------------------
    # ③ 各モデルでコード検索実験を実施
    # ------------------------------
    model_configs = [
        {"name": "microsoft/graphcodebert-base", "class": AutoModelForMaskedLM},
        {"name": "microsoft/codebert-base-mlm", "class": AutoModelForMaskedLM},
        {"name": "Salesforce/codet5p-220m-bimodal", "class": T5ForConditionalGeneration},
        {"name": "Shuu12121/CodeHawks-ModernBERT-1.0", "class": AutoModelForMaskedLM},
        {"name": "Shuu12121/CodeHawks-ModernBERT", "class": AutoModelForMaskedLM},
    ]

    for lang in languages:
        print(f"\n==== 言語: {lang} の評価を開始 ====")
        # データセットをロード後、シャッフルしてランダムサンプルを抽出
        dataset = load_dataset("google/code_x_glue_ct_code_to_text", lang, split="test", trust_remote_code=True)
        dataset = dataset.shuffle(seed=35)
        subset = dataset.select(range(max_examples))

        for config in model_configs:
            model_name = config["name"]
            model_class = config["class"]
            print(f"\n{model_name} を評価します (候補プールサイズ: 100)...")
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = model_class.from_pretrained(model_name)
            model.to(device)

            metrics = evaluate_code_search(model, tokenizer, subset, device,
                                           max_examples=max_examples,
                                           pool_size=100,
                                           query_field="docstring",
                                           code_field="code")
            display_code_search_results(metrics, f"{model_name} - {lang}")


使用デバイス: cuda

【ModernBERT 単体ロード確認】
モデルのロード成功！
ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50000, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=3072, out_features=768, 

README.md:   0%|          | 0.00/26.7k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/144M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

...


microsoft/graphcodebert-base を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== microsoft/graphcodebert-base - go Code Search Evaluation Results ====
MRR:         0.4221
MAP:         0.4221
R-Precision: 0.3230

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.3230    0.3230    0.3230    0.3230    0.3230         0.3230         
5     0.5150    0.3938    0.4239    0.4242    0.5150         0.5150         
10    0.6190    0.4078    0.4577    0.4488    0.6190         0.6190         
50    0.9070    0.4207    0.5203    0.4733    0.9070         0.9070         
100   1.0000    0.4221    0.5355    0.4761    1.0000         1.0000         

microsoft/codebert-base-mlm を評価します (候補プールサイズ: 100)...


Some weights of the model checkpoint at microsoft/codebert-base-mlm were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== microsoft/codebert-base-mlm - go Code Search Evaluation Results ====
MRR:         0.3296
MAP:         0.3296
R-Precision: 0.2500

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.2500    0.2500    0.2500    0.2500    0.2500         0.2500         
5     0.3870    0.2985    0.3205    0.3202    0.3870         0.3870         
10    0.4740    0.3101    0.3486    0.3405    0.4740         0.4740         
50    0.8410    0.3273    0.4294    0.3730    0.8410         0.8410         
100   1.0000    0.3296    0.4554    0.3777    1.0000         1.0000         

Salesforce/codet5p-220m-bimodal を評価します (候補プールサイズ: 100)...


You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal - go Code Search Evaluation Results ====
MRR:         0.8159
MAP:         0.8159
R-Precision: 0.7350

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7350    0.7350    0.7350    0.7350    0.7350         0.7350         
5     0.9230    0.8093    0.8379    0.8397    0.9230         0.9230         
10    0.9520    0.8132    0.8473    0.8466    0.9520         0.9520         
50    0.9970    0.8159    0.8581    0.8516    0.9970         0.9970         
100   1.0000    0.8159    0.8586    0.8517    1.0000         1.0000         

Shuu12121/CodeHawks-ModernBERT-1.0 を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Shuu12121/CodeHawks-ModernBERT-1.0 - go Code Search Evaluation Results ====
MRR:         0.9207
MAP:         0.9207

In [10]:
import torch
import numpy as np
import random
import re
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer, T5ForConditionalGeneration
from sklearn.metrics.pairwise import cosine_similarity

# ---------------------------------------------------
# ユーティリティ関数群
# ---------------------------------------------------

def remove_multiline_strings(code):
    """
    Pythonコード中のマルチライン文字列（""" """ や ''' '''）を削除します。
    """
    pattern = r'("""(.*?)"""|\'\'\'(.*?)\'\'\')'
    return re.sub(pattern, '', code, flags=re.DOTALL)

def get_cls_embedding(model, tokenizer, text, device, max_length=256):
    """
    入力テキストの埋め込みを取得する関数。
    - BERT/CodeBERT系の場合は CLS トークンの埋め込みを使用。
    - CodeT5系の場合は encoder 出力の平均プーリングを使用。
    - ModernBERT のように内部に model がある場合は model.model を利用。
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    # ModernBERT の場合、token_type_ids が不要なので削除する
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if hasattr(model, "model"):
        outputs = model.model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "bert"):
        outputs = model.bert(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "roberta"):
        outputs = model.roberta(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "encoder"):
        # CodeT5系の場合は、CLSトークンが存在しないため平均プーリングを使用
        outputs = model.encoder(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1)
    else:
        outputs = model(**inputs)
        if hasattr(outputs, "last_hidden_state"):
            embedding = outputs.last_hidden_state[:, 0, :]
        elif hasattr(outputs, "hidden_states"):
            embedding = outputs.hidden_states[-1][:, 0, :]
        else:
            raise ValueError("モデル出力にlast_hidden_stateまたはhidden_statesがありません。")

    return embedding.detach().cpu().numpy()


def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def calculate_dcg(ranking):
    dcg = 0.0
    for i, rel in enumerate(ranking):
        dcg += rel / np.log2(i + 2)
    return dcg

def calculate_metrics(sim_matrix, k_values=[1, 5, 10, 50, 100]):
    """
    候補プール内で正解（常にインデックス0が正解と仮定）に基づいて評価指標を計算します。
    """
    num_queries = sim_matrix.shape[0]
    metrics = {
        "mrr": 0.0,
        "map": 0.0,
        "r_precision": 0.0,
        "recall@k": {k: 0.0 for k in k_values},
        "precision@k": {k: 0.0 for k in k_values},
        "ndcg@k": {k: 0.0 for k in k_values},
        "f1@k": {k: 0.0 for k in k_values},
        "success_rate@k": {k: 0.0 for k in k_values},
        "query_coverage@k": {k: 0.0 for k in k_values},
    }

    for i in range(num_queries):
        sims = sim_matrix[i]
        ranked_indices = np.argsort(-sims)
        # 正解は候補プール内の先頭（インデックス0）
        correct_rank = np.where(ranked_indices == 0)[0][0] + 1
        metrics["mrr"] += 1.0 / correct_rank
        metrics["map"] += 1.0 / correct_rank
        metrics["r_precision"] += 1.0 if correct_rank == 1 else 0.0

        for k in k_values:
            if correct_rank <= k:
                prec = 1.0 / correct_rank
                rec = 1.0
            else:
                prec = 0.0
                rec = 0.0
            f1 = calculate_f1(prec, rec)
            metrics["precision@k"][k] += prec
            metrics["recall@k"][k] += rec
            metrics["f1@k"][k] += f1

            ideal_ranking = [1.0] + [0.0] * (k - 1)
            actual_ranking = [1.0 if j == 0 else 0.0 for j in ranked_indices[:k]]
            idcg = calculate_dcg(ideal_ranking)
            dcg = calculate_dcg(actual_ranking)
            metrics["ndcg@k"][k] += dcg / idcg if idcg > 0 else 0.0

            metrics["success_rate@k"][k] += 1.0 if correct_rank <= k else 0.0
            metrics["query_coverage@k"][k] += 1.0 if correct_rank <= k else 0.0

    metrics["mrr"] /= num_queries
    metrics["map"] /= num_queries
    metrics["r_precision"] /= num_queries
    for k in k_values:
        metrics["precision@k"][k] /= num_queries
        metrics["recall@k"][k] /= num_queries
        metrics["ndcg@k"][k] /= num_queries
        metrics["f1@k"][k] /= num_queries
        metrics["success_rate@k"][k] /= num_queries
        metrics["query_coverage@k"][k] /= num_queries
    return metrics

def display_code_search_results(metrics, model_name):
    """
    評価結果を見やすいテーブル形式で表示します。
    """
    print(f"\n==== {model_name} Code Search Evaluation Results ====")
    print(f"MRR:         {metrics['mrr']:.4f}")
    print(f"MAP:         {metrics['map']:.4f}")
    print(f"R-Precision: {metrics['r_precision']:.4f}\n")

    header = "{:<6}{:<10}{:<10}{:<10}{:<10}{:<15}{:<15}"
    print(header.format("K", "Recall", "Precision", "NDCG", "F1", "Success Rate", "Query Cov."))
    print("-" * 76)
    for k in sorted(metrics["recall@k"].keys()):
        recall = metrics["recall@k"][k]
        precision = metrics["precision@k"][k]
        ndcg = metrics["ndcg@k"][k]
        f1 = metrics["f1@k"][k]
        success_rate = metrics["success_rate@k"][k]
        query_cov = metrics["query_coverage@k"][k]
        print(header.format(k,
                            f"{recall:.4f}",
                            f"{precision:.4f}",
                            f"{ndcg:.4f}",
                            f"{f1:.4f}",
                            f"{success_rate:.4f}",
                            f"{query_cov:.4f}"))

def evaluate_code_search(model, tokenizer, dataset, device, max_examples=100, pool_size=100,
                         query_field="docstring", code_field="code"):
    """
    指定したデータセット上でコード検索タスクを実施し、評価指標を計算します。
    各サンプルに対して、正解は候補プール内の先頭（自身）と仮定しています。
    """
    num_examples = min(len(dataset), max_examples)
    all_codes = [remove_multiline_strings(dataset[i][code_field]) for i in range(num_examples)]
    queries = [dataset[i][query_field] for i in range(num_examples)]

    # 各コードの埋め込み計算
    all_code_embeddings = []
    print("データセット全体のコード埋め込みを計算中...")
    for code in all_codes:
        emb = get_cls_embedding(model, tokenizer, code, device)
        all_code_embeddings.append(emb)
    all_code_embeddings = np.concatenate(all_code_embeddings, axis=0)
    print("コード埋め込み計算完了.")

    # 各クエリに対し候補プール内の類似度計算
    sim_matrices = []
    print("候補コードプールを作成し、類似度計算中...")
    for i in range(num_examples):
        query_embedding = get_cls_embedding(model, tokenizer, queries[i], device).reshape(1, -1)
        candidate_indices = [i] + random.sample([j for j in range(num_examples) if j != i], pool_size - 1)
        candidate_embeddings = all_code_embeddings[candidate_indices]
        sims = cosine_similarity(query_embedding, candidate_embeddings)[0]
        sim_matrices.append(sims)
    sim_matrix = np.array(sim_matrices)
    print("類似度計算完了.")

    metrics = calculate_metrics(sim_matrix)
    return metrics

# ---------------------------------------------------
# メイン処理：各モデル・各言語でコード検索実験を実施
# ---------------------------------------------------

if __name__ == "__main__":
    # Python の乱数シードを設定（候補サンプルの抽出などに影響します）
    random.seed(35)
    np.random.seed(35)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用デバイス: {device}")

    # ------------------------------
    # ① ModernBERT 単体ロード確認
    # ------------------------------
    repo_name = "Shuu12121/CodeMorph-ModernBERT"
    model_demo = AutoModelForMaskedLM.from_pretrained(repo_name)
    tokenizer_demo = AutoTokenizer.from_pretrained(repo_name)
    model_demo.to(device)
    print("\n【ModernBERT 単体ロード確認】")
    print("モデルのロード成功！")
    print(model_demo)

    # ------------------------------
    # ② 評価データセットの言語一覧（例）
    # ------------------------------
    languages = ["python", "java", "javascript", "php", "ruby", "go"]
    max_examples = 1000  # 各言語ごとに先頭100サンプルを利用（ランダムサンプリング後）

    # ------------------------------
    # ③ 各モデルでコード検索実験を実施
    # ------------------------------
    model_configs = [
        {"name": "Salesforce/codet5p-220m-bimodal", "class": T5ForConditionalGeneration},
        {"name": "Shuu12121/CodeHawks-ModernBERT", "class": AutoModelForMaskedLM},
        {"name": "Shuu12121/CodeHawks-ModernBERT-1.0", "class": AutoModelForMaskedLM},
    ]

    for lang in languages:
        print(f"\n==== 言語: {lang} の評価を開始 ====")
        # データセットをロード後、シャッフルしてランダムサンプルを抽出
        dataset = load_dataset("code-search-net/code_search_net", lang, split="test", trust_remote_code=True)
        dataset = dataset.shuffle(seed=35)
        subset = dataset.select(range(max_examples))

        for config in model_configs:
            model_name = config["name"]
            model_class = config["class"]
            print(f"\n{model_name} を評価します (候補プールサイズ: 100)...")
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = model_class.from_pretrained(model_name)
            model.to(device)

            metrics = evaluate_code_search(model, tokenizer, subset, device,
                                           max_examples=max_examples,
                                           pool_size=100,
                                           query_field="func_documentation_string",
                                           code_field="func_code_string")
            display_code_search_results(metrics, f"{model_name} - {lang}")


使用デバイス: cuda

【ModernBERT 単体ロード確認】
モデルのロード成功！
ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50000, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=3072, out_features=768, 

You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal - python Code Search Evaluation Results ====
MRR:         0.7944
MAP:         0.7944
R-Precision: 0.7190

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7190    0.7190    0.7190    0.7190    0.7190         0.7190         
5     0.8930    0.7866    0.8133    0.8145    0.8930         0.8930         
10    0.9280    0.7912    0.8246    0.8227    0.9280         0.9280         
50    0.9910    0.7942    0.8385    0.8284    0.9910         0.9910         
100   1.0000    0.7944    0.8400    0.8287    1.0000         1.0000         

Shuu12121/CodeHawks-ModernBERT を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Shuu12121/CodeHawks-ModernBERT - python Code Search Evaluation Results ====
MRR:         0.8561
MAP:         0.8561

You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal - java Code Search Evaluation Results ====
MRR:         0.7854
MAP:         0.7854
R-Precision: 0.7110

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7110    0.7110    0.7110    0.7110    0.7110         0.7110         
5     0.8780    0.7781    0.8033    0.8052    0.8780         0.8780         
10    0.9100    0.7822    0.8135    0.8124    0.9100         0.9100         
50    0.9740    0.7850    0.8273    0.8178    0.9740         0.9740         
100   1.0000    0.7854    0.8316    0.8186    1.0000         1.0000         

Shuu12121/CodeHawks-ModernBERT を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Shuu12121/CodeHawks-ModernBERT - java Code Search Evaluation Results ====
MRR:         0.7148
MAP:         0.7148
R-P

...

データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal - php Code Search Evaluation Results ====
MRR:         0.7857
MAP:         0.7857
R-Precision: 0.7230

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7230    0.7230    0.7230    0.7230    0.7230         0.7230         
5     0.8660    0.7775    0.7997    0.8005    0.8660         0.8660         
10    0.8980    0.7819    0.8101    0.8081    0.8980         0.8980         
50    0.9660    0.7852    0.8252    0.8144    0.9660         0.9660         
100   1.0000    0.7857    0.8308    0.8153    1.0000         1.0000         

Shuu12121/CodeHawks-ModernBERT を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Shuu12121/CodeHawks-ModernBERT - php Code Search Evaluation Results ====
MRR:         0.7504
MAP:         0.7504
R-Pre

You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal - ruby Code Search Evaluation Results ====
MRR:         0.7137
MAP:         0.7137
R-Precision: 0.6260

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.6260    0.6260    0.6260    0.6260    0.6260         0.6260         
5     0.8270    0.7026    0.7337    0.7346    0.8270         0.8270         
10    0.8690    0.7081    0.7471    0.7443    0.8690         0.8690         
50    0.9670    0.7132    0.7695    0.7540    0.9670         0.9670         
100   1.0000    0.7137    0.7749    0.7550    1.0000         1.0000         

Shuu12121/CodeHawks-ModernBERT を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Shuu12121/CodeHawks-ModernBERT - ruby Code Search Evaluation Results ====
MRR:         0.7300
MAP:         0.7300
R-P

You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal - go Code Search Evaluation Results ====
MRR:         0.7527
MAP:         0.7527
R-Precision: 0.6550

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.6550    0.6550    0.6550    0.6550    0.6550         0.6550         
5     0.8790    0.7404    0.7751    0.7763    0.8790         0.8790         
10    0.9440    0.7496    0.7966    0.7923    0.9440         0.9440         
50    0.9970    0.7526    0.8091    0.7980    0.9970         0.9970         
100   1.0000    0.7527    0.8096    0.7981    1.0000         1.0000         

Shuu12121/CodeHawks-ModernBERT を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Shuu12121/CodeHawks-ModernBERT - go Code Search Evaluation Results ====
MRR:         0.8219
MAP:         0.8219
R-Preci

In [12]:
import torch
import numpy as np
import random
import re
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer, T5ForConditionalGeneration
from sklearn.metrics.pairwise import cosine_similarity

# ---------------------------------------------------
# ユーティリティ関数群
# ---------------------------------------------------

def remove_multiline_strings(code):
    """
    Pythonコード中のマルチライン文字列（""" """ や ''' '''）を削除します。
    """
    pattern = r'("""(.*?)"""|\'\'\'(.*?)\'\'\')'
    return re.sub(pattern, '', code, flags=re.DOTALL)

def get_cls_embedding(model, tokenizer, text, device, max_length=256):
    """
    入力テキストの埋め込みを取得する関数。
    - BERT/CodeBERT系の場合は CLS トークンの埋め込みを使用。
    - CodeT5系の場合は encoder 出力の平均プーリングを使用。
    - ModernBERT のように内部に model がある場合は model.model を利用。
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    # ModernBERT の場合、token_type_ids が不要なので削除する
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if hasattr(model, "model"):
        outputs = model.model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "bert"):
        outputs = model.bert(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "roberta"):
        outputs = model.roberta(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "encoder"):
        # CodeT5系の場合は、CLSトークンが存在しないため平均プーリングを使用
        outputs = model.encoder(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1)
    else:
        outputs = model(**inputs)
        if hasattr(outputs, "last_hidden_state"):
            embedding = outputs.last_hidden_state[:, 0, :]
        elif hasattr(outputs, "hidden_states"):
            embedding = outputs.hidden_states[-1][:, 0, :]
        else:
            raise ValueError("モデル出力にlast_hidden_stateまたはhidden_statesがありません。")

    return embedding.detach().cpu().numpy()


def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def calculate_dcg(ranking):
    dcg = 0.0
    for i, rel in enumerate(ranking):
        dcg += rel / np.log2(i + 2)
    return dcg

def calculate_metrics(sim_matrix, k_values=[1, 5, 10, 50, 100]):
    """
    候補プール内で正解（常にインデックス0が正解と仮定）に基づいて評価指標を計算します。
    """
    num_queries = sim_matrix.shape[0]
    metrics = {
        "mrr": 0.0,
        "map": 0.0,
        "r_precision": 0.0,
        "recall@k": {k: 0.0 for k in k_values},
        "precision@k": {k: 0.0 for k in k_values},
        "ndcg@k": {k: 0.0 for k in k_values},
        "f1@k": {k: 0.0 for k in k_values},
        "success_rate@k": {k: 0.0 for k in k_values},
        "query_coverage@k": {k: 0.0 for k in k_values},
    }

    for i in range(num_queries):
        sims = sim_matrix[i]
        ranked_indices = np.argsort(-sims)
        # 正解は候補プール内の先頭（インデックス0）
        correct_rank = np.where(ranked_indices == 0)[0][0] + 1
        metrics["mrr"] += 1.0 / correct_rank
        metrics["map"] += 1.0 / correct_rank
        metrics["r_precision"] += 1.0 if correct_rank == 1 else 0.0

        for k in k_values:
            if correct_rank <= k:
                prec = 1.0 / correct_rank
                rec = 1.0
            else:
                prec = 0.0
                rec = 0.0
            f1 = calculate_f1(prec, rec)
            metrics["precision@k"][k] += prec
            metrics["recall@k"][k] += rec
            metrics["f1@k"][k] += f1

            ideal_ranking = [1.0] + [0.0] * (k - 1)
            actual_ranking = [1.0 if j == 0 else 0.0 for j in ranked_indices[:k]]
            idcg = calculate_dcg(ideal_ranking)
            dcg = calculate_dcg(actual_ranking)
            metrics["ndcg@k"][k] += dcg / idcg if idcg > 0 else 0.0

            metrics["success_rate@k"][k] += 1.0 if correct_rank <= k else 0.0
            metrics["query_coverage@k"][k] += 1.0 if correct_rank <= k else 0.0

    metrics["mrr"] /= num_queries
    metrics["map"] /= num_queries
    metrics["r_precision"] /= num_queries
    for k in k_values:
        metrics["precision@k"][k] /= num_queries
        metrics["recall@k"][k] /= num_queries
        metrics["ndcg@k"][k] /= num_queries
        metrics["f1@k"][k] /= num_queries
        metrics["success_rate@k"][k] /= num_queries
        metrics["query_coverage@k"][k] /= num_queries
    return metrics

def display_code_search_results(metrics, model_name):
    """
    評価結果を見やすいテーブル形式で表示します。
    """
    print(f"\n==== {model_name} Code Search Evaluation Results ====")
    print(f"MRR:         {metrics['mrr']:.4f}")
    print(f"MAP:         {metrics['map']:.4f}")
    print(f"R-Precision: {metrics['r_precision']:.4f}\n")

    header = "{:<6}{:<10}{:<10}{:<10}{:<10}{:<15}{:<15}"
    print(header.format("K", "Recall", "Precision", "NDCG", "F1", "Success Rate", "Query Cov."))
    print("-" * 76)
    for k in sorted(metrics["recall@k"].keys()):
        recall = metrics["recall@k"][k]
        precision = metrics["precision@k"][k]
        ndcg = metrics["ndcg@k"][k]
        f1 = metrics["f1@k"][k]
        success_rate = metrics["success_rate@k"][k]
        query_cov = metrics["query_coverage@k"][k]
        print(header.format(k,
                            f"{recall:.4f}",
                            f"{precision:.4f}",
                            f"{ndcg:.4f}",
                            f"{f1:.4f}",
                            f"{success_rate:.4f}",
                            f"{query_cov:.4f}"))

def evaluate_code_search(model, tokenizer, dataset, device, max_examples=100, pool_size=100,
                         query_field="docstring", code_field="code"):
    """
    指定したデータセット上でコード検索タスクを実施し、評価指標を計算します。
    各サンプルに対して、正解は候補プール内の先頭（自身）と仮定しています。
    """
    num_examples = min(len(dataset), max_examples)
    all_codes = [remove_multiline_strings(dataset[i][code_field]) for i in range(num_examples)]
    queries = [dataset[i][query_field] for i in range(num_examples)]

    # 各コードの埋め込み計算
    all_code_embeddings = []
    print("データセット全体のコード埋め込みを計算中...")
    for code in all_codes:
        emb = get_cls_embedding(model, tokenizer, code, device)
        all_code_embeddings.append(emb)
    all_code_embeddings = np.concatenate(all_code_embeddings, axis=0)
    print("コード埋め込み計算完了.")

    # 各クエリに対し候補プール内の類似度計算
    sim_matrices = []
    print("候補コードプールを作成し、類似度計算中...")
    for i in range(num_examples):
        query_embedding = get_cls_embedding(model, tokenizer, queries[i], device).reshape(1, -1)
        candidate_indices = [i] + random.sample([j for j in range(num_examples) if j != i], pool_size - 1)
        candidate_embeddings = all_code_embeddings[candidate_indices]
        sims = cosine_similarity(query_embedding, candidate_embeddings)[0]
        sim_matrices.append(sims)
    sim_matrix = np.array(sim_matrices)
    print("類似度計算完了.")

    metrics = calculate_metrics(sim_matrix)
    return metrics

# ---------------------------------------------------
# メイン処理：各モデルでコード検索実験を実施
# ---------------------------------------------------

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用デバイス: {device}")

    # ------------------------------
    # ① ModernBERT 単体ロード確認
    # ------------------------------
    repo_name = "Shuu12121/CodeMorph-ModernBERT"
    model_demo = AutoModelForMaskedLM.from_pretrained(repo_name)
    tokenizer_demo = AutoTokenizer.from_pretrained(repo_name)
    model_demo.to(device)
    print("\n【ModernBERT 単体ロード確認】")
    print("モデルのロード成功！")
    print(model_demo)

    # ------------------------------
    # ② 評価データセットのロード
    # ------------------------------
    print("\ngoogle/code_x_glue_tc_nl_code_search_adv データセット (Test) をロードします...")
    tc_dataset = load_dataset("google/code_x_glue_tc_nl_code_search_adv", split="test", trust_remote_code=True)
    dataset = tc_dataset.shuffle(seed=35)
    max_examples= 19210
    subset = dataset.select(range(max_examples))

    # ------------------------------
    # ③ 各モデルでコード検索実験を実施
    # ------------------------------
    model_configs = [
        {"name": "Salesforce/codet5p-220m-bimodal", "class": T5ForConditionalGeneration},
        {"name": "Shuu12121/CodeHawks-ModernBERT", "class": AutoModelForMaskedLM},
        {"name": "Shuu12121/CodeHawks-ModernBERT-1.0", "class": AutoModelForMaskedLM},
    ]

    for config in model_configs:
        model_name = config["name"]
        model_class = config["class"]
        print(f"\n{model_name} を評価します (候補プールサイズ: 100)...")
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = model_class.from_pretrained(model_name)
        model.to(device)

        metrics = evaluate_code_search(model, tokenizer, subset, device,
                                       max_examples=19210,
                                       pool_size=100,
                                       query_field="docstring",
                                       code_field="code")
        display_code_search_results(metrics, model_name)


使用デバイス: cuda

【ModernBERT 単体ロード確認】
モデルのロード成功！
ModernBertForMaskedLM(
  (model): ModernBertModel(
    (embeddings): ModernBertEmbeddings(
      (tok_embeddings): Embedding(50000, 768, padding_idx=0)
      (norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (layers): ModuleList(
      (0): ModernBertEncoderLayer(
        (attn_norm): Identity()
        (attn): ModernBertAttention(
          (Wqkv): Linear(in_features=768, out_features=2304, bias=False)
          (rotary_emb): ModernBertRotaryEmbedding()
          (Wo): Linear(in_features=768, out_features=768, bias=False)
          (out_drop): Identity()
        )
        (mlp_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): ModernBertMLP(
          (Wi): Linear(in_features=768, out_features=6144, bias=False)
          (act): GELUActivation()
          (drop): Dropout(p=0.0, inplace=False)
          (Wo): Linear(in_features=3072, out_features=768, 

You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal Code Search Evaluation Results ====
MRR:         0.5348
MAP:         0.5348
R-Precision: 0.4240

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.4240    0.4240    0.4240    0.4240    0.4240         0.4240         
5     0.6590    0.5118    0.5485    0.5492    0.6590         0.6590         
10    0.7545    0.5246    0.5794    0.5716    0.7545         0.7545         
50    0.9466    0.5340    0.6223    0.5894    0.9466         0.9466         
100   1.0000    0.5348    0.6311    0.5910    1.0000         1.0000         

Shuu12121/CodeHawks-ModernBERT を評価します (候補プールサイズ: 100)...
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Shuu12121/CodeHawks-ModernBERT Code Search Evaluation Results ====
MRR:         0.6767
MAP:         0.6767
R-Precision: 0.58

In [13]:
import torch
import numpy as np
import random
import re
from datasets import load_dataset
from transformers import AutoModelForMaskedLM, AutoTokenizer, T5ForConditionalGeneration
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# ---------------------------------------------------
# ユーティリティ関数群
# ---------------------------------------------------

def remove_multiline_strings(code):
    """
    Pythonコード中のマルチライン文字列 (""" """ や ''' ''' ) を削除します。
    """
    pattern = r'("""(.*?)"""|\'\'\'(.*?)\'\'\')'
    return re.sub(pattern, '', code, flags=re.DOTALL)

def get_cls_embedding(model, tokenizer, text, device, max_length=256):
    """
    入力テキストの埋め込みを取得する関数。
    - BERT/CodeBERT系の場合は CLS トークンの埋め込みを使用。
    - CodeT5系の場合は encoder 出力の平均プーリングを使用。
    - ModernBERT のように内部に model がある場合は model.model を利用。
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=max_length)
    if "token_type_ids" in inputs:
        inputs.pop("token_type_ids")
    inputs = {k: v.to(device) for k, v in inputs.items()}

    if hasattr(model, "model"):
        outputs = model.model(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "bert"):
        outputs = model.bert(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "roberta"):
        outputs = model.roberta(**inputs)
        embedding = outputs.last_hidden_state[:, 0, :]
    elif hasattr(model, "encoder"):
        outputs = model.encoder(**inputs)
        embedding = outputs.last_hidden_state.mean(dim=1)
    else:
        outputs = model(**inputs)
        if hasattr(outputs, "last_hidden_state"):
            embedding = outputs.last_hidden_state[:, 0, :]
        elif hasattr(outputs, "hidden_states"):
            embedding = outputs.hidden_states[-1][:, 0, :]
        else:
            raise ValueError("モデル出力にlast_hidden_stateまたはhidden_statesがありません。")
    return embedding.detach().cpu().numpy()

def calculate_f1(precision, recall):
    if precision + recall == 0:
        return 0.0
    return 2 * (precision * recall) / (precision + recall)

def calculate_dcg(ranking):
    dcg = 0.0
    for i, rel in enumerate(ranking):
        dcg += rel / np.log2(i + 2)
    return dcg

def calculate_metrics(sim_matrix, k_values=[1, 5, 10, 50, 100]):
    """
    候補プール内で正解（常にインデックス0が正解と仮定）に基づいて評価指標を計算します。
    """
    num_queries = sim_matrix.shape[0]
    metrics = {
        "mrr": 0.0,
        "map": 0.0,
        "r_precision": 0.0,
        "recall@k": {k: 0.0 for k in k_values},
        "precision@k": {k: 0.0 for k in k_values},
        "ndcg@k": {k: 0.0 for k in k_values},
        "f1@k": {k: 0.0 for k in k_values},
        "success_rate@k": {k: 0.0 for k in k_values},
        "query_coverage@k": {k: 0.0 for k in k_values},
    }

    for i in range(num_queries):
        sims = sim_matrix[i]
        ranked_indices = np.argsort(-sims)
        # 正解は候補プール内の先頭（インデックス0）
        correct_rank = np.where(ranked_indices == 0)[0][0] + 1
        metrics["mrr"] += 1.0 / correct_rank
        metrics["map"] += 1.0 / correct_rank
        metrics["r_precision"] += 1.0 if correct_rank == 1 else 0.0

        for k in k_values:
            if correct_rank <= k:
                prec = 1.0 / correct_rank
                rec = 1.0
            else:
                prec = 0.0
                rec = 0.0
            f1 = calculate_f1(prec, rec)
            metrics["precision@k"][k] += prec
            metrics["recall@k"][k] += rec
            metrics["f1@k"][k] += f1

            ideal_ranking = [1.0] + [0.0] * (k - 1)
            actual_ranking = [1.0 if j == 0 else 0.0 for j in ranked_indices[:k]]
            idcg = calculate_dcg(ideal_ranking)
            dcg = calculate_dcg(actual_ranking)
            metrics["ndcg@k"][k] += dcg / idcg if idcg > 0 else 0.0

            metrics["success_rate@k"][k] += 1.0 if correct_rank <= k else 0.0
            metrics["query_coverage@k"][k] += 1.0 if correct_rank <= k else 0.0

    metrics["mrr"] /= num_queries
    metrics["map"] /= num_queries
    metrics["r_precision"] /= num_queries
    for k in k_values:
        metrics["precision@k"][k] /= num_queries
        metrics["recall@k"][k] /= num_queries
        metrics["ndcg@k"][k] /= num_queries
        metrics["f1@k"][k] /= num_queries
        metrics["success_rate@k"][k] /= num_queries
        metrics["query_coverage@k"][k] /= num_queries
    return metrics

def display_code_search_results(metrics, model_name):
    """
    評価結果を見やすいテーブル形式で表示します。
    """
    print(f"\n==== {model_name} Code Search Evaluation Results ====")
    print(f"MRR:         {metrics['mrr']:.4f}")
    print(f"MAP:         {metrics['map']:.4f}")
    print(f"R-Precision: {metrics['r_precision']:.4f}\n")

    header = "{:<6}{:<10}{:<10}{:<10}{:<10}{:<15}{:<15}"
    print(header.format("K", "Recall", "Precision", "NDCG", "F1", "Success Rate", "Query Cov."))
    print("-" * 76)
    for k in sorted(metrics["recall@k"].keys()):
        recall = metrics["recall@k"][k]
        precision = metrics["precision@k"][k]
        ndcg = metrics["ndcg@k"][k]
        f1 = metrics["f1@k"][k]
        success_rate = metrics["success_rate@k"][k]
        query_cov = metrics["query_coverage@k"][k]
        print(header.format(k,
                            f"{recall:.4f}",
                            f"{precision:.4f}",
                            f"{ndcg:.4f}",
                            f"{f1:.4f}",
                            f"{success_rate:.4f}",
                            f"{query_cov:.4f}"))

def evaluate_code_search(model, tokenizer, dataset, device, max_examples=100, pool_size=100,
                         query_field="docstring", code_field="code"):
    """
    指定したデータセット上でコード検索タスクを実施し、評価指標を計算します。
    各サンプルに対して、正解は候補プール内の先頭（自身）と仮定しています。
    """
    num_examples = min(len(dataset), max_examples)
    all_codes = [remove_multiline_strings(dataset[i][code_field]) for i in range(num_examples)]
    queries = [dataset[i][query_field] for i in range(num_examples)]

    # 各コードの埋め込み計算
    all_code_embeddings = []
    print("データセット全体のコード埋め込みを計算中...")
    for code in all_codes:
        emb = get_cls_embedding(model, tokenizer, code, device)
        all_code_embeddings.append(emb)
    all_code_embeddings = np.concatenate(all_code_embeddings, axis=0)
    print("コード埋め込み計算完了.")

    # 各クエリに対して候補プール内の類似度計算
    sim_matrices = []
    print("候補コードプールを作成し、類似度計算中...")
    for i in range(num_examples):
        query_embedding = get_cls_embedding(model, tokenizer, queries[i], device).reshape(1, -1)
        candidate_indices = [i] + random.sample([j for j in range(num_examples) if j != i], pool_size - 1)
        candidate_embeddings = all_code_embeddings[candidate_indices]
        sims = cosine_similarity(query_embedding, candidate_embeddings)[0]
        sim_matrices.append(sims)
    sim_matrix = np.array(sim_matrices)
    print("類似度計算完了.")

    metrics = calculate_metrics(sim_matrix)
    return metrics

# ---------------------------------------------------
# 各モデル・各シード・各言語で実験を実施する
# ---------------------------------------------------

if __name__ == "__main__":
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"使用デバイス: {device}")

    # シード値リスト（5種類）
    seeds = [42, 123, 456, 789, 1024]
    # 対象言語リスト
    languages = ["python", "java", "javascript", "php", "ruby", "go"]
    max_examples = 1000  # 各言語ごとのサンプル数
    pool_size = 100      # 候補プールのサイズ

    # モデル設定（各モデルごとに実験）
    model_configs = [
        {"name": "Salesforce/codet5p-220m-bimodal", "class": T5ForConditionalGeneration},
        {"name": "Shuu12121/CodeHawks-ModernBERT", "class": AutoModelForMaskedLM},
        {"name": "Shuu12121/CodeHawks-ModernBERT-1.0", "class": AutoModelForMaskedLM},
    ]

    # 結果を格納する辞書
    # 構造: all_results[model_name][seed][language] = metrics
    all_results = {}

    for config in model_configs:
        model_name = config["name"]
        all_results[model_name] = {}
        # モデル・トークナイザーは各シードで共通で利用する場合もありますが、
        # ここでは各シードで再ロードしてシードによる変動を反映させます
        for seed in seeds:
            # シードの設定
            random.seed(seed)
            np.random.seed(seed)
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.cuda.manual_seed_all(seed)

            # モデルとトークナイザーのロード
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = config["class"].from_pretrained(model_name)
            model.to(device)

            all_results[model_name][seed] = {}
            for lang in languages:
                print(f"\n===== Model: {model_name} | Seed: {seed} | Language: {lang} =====")
                # 各言語ごとにデータセットをロード（テスト分割）し、サブセットを抽出
                dataset = load_dataset("google/code_x_glue_ct_code_to_text", lang, split="test", trust_remote_code=True)
                dataset = dataset.shuffle(seed=seed)
                subset = dataset.select(range(max_examples))

                metrics = evaluate_code_search(model, tokenizer, subset, device,
                                               max_examples=max_examples, pool_size=pool_size,
                                               query_field="docstring", code_field="code")
                display_code_search_results(metrics, f"{model_name} | Seed: {seed} | Lang: {lang}")
                all_results[model_name][seed][lang] = metrics


使用デバイス: cuda


You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.



===== Model: Salesforce/codet5p-220m-bimodal | Seed: 42 | Language: python =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal | Seed: 42 | Lang: python Code Search Evaluation Results ====
MRR:         0.8227
MAP:         0.8227
R-Precision: 0.7590

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7590    0.7590    0.7590    0.7590    0.7590         0.7590         
5     0.8970    0.8156    0.8361    0.8379    0.8970         0.8970         
10    0.9270    0.8195    0.8458    0.8448    0.9270         0.9270         
50    0.9880    0.8225    0.8595    0.8505    0.9880         0.9880         
100   1.0000    0.8227    0.8615    0.8509    1.0000         1.0000         

===== Model: Salesforce/codet5p-220m-bimodal | Seed: 42 | Language: java =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.


You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.



===== Model: Salesforce/codet5p-220m-bimodal | Seed: 123 | Language: python =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal | Seed: 123 | Lang: python Code Search Evaluation Results ====
MRR:         0.8268
MAP:         0.8268
R-Precision: 0.7580

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7580    0.7580    0.7580    0.7580    0.7580         0.7580         
5     0.9110    0.8207    0.8435    0.8454    0.9110         0.9110         
10    0.9360    0.8240    0.8516    0.8513    0.9360         0.9360         
50    0.9920    0.8266    0.8639    0.8562    0.9920         0.9920         
100   1.0000    0.8268    0.8652    0.8565    1.0000         1.0000         

===== Model: Salesforce/codet5p-220m-bimodal | Seed: 123 | Language: java =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完

...


===== Model: Salesforce/codet5p-220m-bimodal | Seed: 456 | Language: python =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal | Seed: 456 | Lang: python Code Search Evaluation Results ====
MRR:         0.8194
MAP:         0.8194
R-Precision: 0.7490

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7490    0.7490    0.7490    0.7490    0.7490         0.7490         
5     0.9100    0.8128    0.8372    0.8387    0.9100         0.9100         
10    0.9410    0.8171    0.8474    0.8461    0.9410         0.9410         
50    0.9860    0.8191    0.8572    0.8501    0.9860         0.9860         
100   1.0000    0.8194    0.8596    0.8505    1.0000         1.0000         

===== Model: Salesforce/codet5p-220m-bimodal | Seed: 456 | Language: java =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完

You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.



===== Model: Salesforce/codet5p-220m-bimodal | Seed: 789 | Language: python =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal | Seed: 789 | Lang: python Code Search Evaluation Results ====
MRR:         0.8248
MAP:         0.8248
R-Precision: 0.7600

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7600    0.7600    0.7600    0.7600    0.7600         0.7600         
5     0.9020    0.8177    0.8390    0.8407    0.9020         0.9020         
10    0.9320    0.8216    0.8486    0.8476    0.9320         0.9320         
50    0.9900    0.8246    0.8617    0.8532    0.9900         0.9900         
100   1.0000    0.8248    0.8634    0.8535    1.0000         1.0000         

===== Model: Salesforce/codet5p-220m-bimodal | Seed: 789 | Language: java =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完

You are using a model of type codet5p_bimodal to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.



===== Model: Salesforce/codet5p-220m-bimodal | Seed: 1024 | Language: python =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度計算完了.

==== Salesforce/codet5p-220m-bimodal | Seed: 1024 | Lang: python Code Search Evaluation Results ====
MRR:         0.8277
MAP:         0.8277
R-Precision: 0.7600

K     Recall    Precision NDCG      F1        Success Rate   Query Cov.     
----------------------------------------------------------------------------
1     0.7600    0.7600    0.7600    0.7600    0.7600         0.7600         
5     0.9060    0.8206    0.8422    0.8442    0.9060         0.9060         
10    0.9360    0.8247    0.8520    0.8514    0.9360         0.9360         
50    0.9920    0.8275    0.8646    0.8568    0.9920         0.9920         
100   1.0000    0.8277    0.8660    0.8571    1.0000         1.0000         

===== Model: Salesforce/codet5p-220m-bimodal | Seed: 1024 | Language: java =====
データセット全体のコード埋め込みを計算中...
コード埋め込み計算完了.
候補コードプールを作成し、類似度計算中...
類似度

In [14]:
# prompt: 切断する

from google.colab import runtime
runtime.unassign()
